# Load the data

In [2]:
import pandas as pd
A = pd.read_csv("/users/ravi/Desktop/datasets/training_set_loan.csv")

In [3]:
A.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
from PM88 import replacer
replacer(A)

In [5]:
Y = A[["Loan_Status"]]
X = A.drop(labels=["Loan_ID","Loan_Status"],axis=1)

# use label encoder

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y.Loan_Status = le.fit_transform(Y)

C:\Users\Ravi\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Ravi\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# Preprocessing

In [7]:
from PM88 import preprocessing
Xnew = preprocessing(X)

# split the data

In [9]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

In [11]:
xtrain.shape[1]

20

# Lets create an ANN with only 1 hidden layer(untuned)

In [13]:
from keras.layers import Dense,Dropout
from keras.models import Sequential

In [14]:
nn = Sequential()
nn.add(Dense(128,input_dim=xtrain.shape[1]))
nn.add(Dense(20))
nn.add(Dropout(0.1))
nn.add(Dense(1,activation="sigmoid"))

In [15]:
nn.compile(loss="binary_crossentropy",metrics="accuracy")
nn.fit(xtrain,ytrain,epochs=50,validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 2s 26ms/step - loss: 0.5429 - accuracy: 0.7628 - val_loss: 0.4899 - val_accuracy: 0.8182
Epoch 2/50
13/13 [==============================] - 0s 4ms/step - loss: 0.4772 - accuracy: 0.8036 - val_loss: 0.4808 - val_accuracy: 0.8182
Epoch 3/50
13/13 [==============================] - 0s 4ms/step - loss: 0.4725 - accuracy: 0.8061 - val_loss: 0.4876 - val_accuracy: 0.8182
Epoch 4/50
13/13 [==============================] - 0s 4ms/step - loss: 0.4611 - accuracy: 0.8061 - val_loss: 0.4909 - val_accuracy: 0.8182
Epoch 5/50
13/13 [==============================] - 0s 3ms/step - loss: 0.4466 - accuracy: 0.8061 - val_loss: 0.4886 - val_accuracy: 0.8081
Epoch 6/50
13/13 [==============================] - 0s 3ms/step - loss: 0.4621 - accuracy: 0.7908 - val_loss: 0.4894 - val_accuracy: 0.8182
Epoch 7/50
13/13 [==============================] - 0s 3ms/step - loss: 0.4504 - accuracy: 0.8112 - val_loss: 0.4979 - val_accuracy: 0.8182
Epoch 8/50
13/13 [=

In [16]:
val_accuracy: 0.8138

# Keras Tuner:¶


In [17]:
def model_builder(hp):
    nn = Sequential()
    nn.add(Dense(128,input_dim=xtrain.shape[1]))
    x = hp.Int("units",min_value=32,max_value=256,step=32)
    nn.add(Dense(units=x))
    y = hp.Choice("rate",values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7])
    nn.add(Dropout(rate=y))
    nn.add(Dense(1,activation="sigmoid"))
    nn.compile(loss="binary_crossentropy",metrics="accuracy")
    return nn

In [19]:
import keras_tuner as kt
tuner = kt.Hyperband(model_builder,
            objective="val_accuracy",
            max_epochs=10,
            directory="/users/ravi/Desktop/KT_NN4",
            project_name="Keras_TUNER")

In [20]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor="val_loss",patience=25)

In [21]:
tuner.search(xtrain,ytrain,epochs=500,validation_split=0.2,callbacks=[es])

Trial 30 Complete [00h 00m 01s]
val_accuracy: 0.808080792427063

Best val_accuracy So Far: 0.8282828330993652
Total elapsed time: 00h 00m 46s
INFO:tensorflow:Oracle triggered exit


In [22]:
tuner.get_best_hyperparameters()[0].get("rate")


0.6

# testing set

In [25]:

B = pd.read_csv("/users/ravi/Desktop/datasets/testing_set_loan.csv")


In [26]:
B.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [27]:
xtest = B.drop(labels=["Loan_ID"],axis=1)

In [29]:
xtest.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [30]:
replacer(xtest)

In [31]:
xtest = preprocessing(xtest)


In [32]:
xtest.shape


(367, 20)

In [40]:
#nn.predict(xtest)

In [45]:
le.inverse_transform(nn.predict(xtest))

In [44]:
B["Predicted_loan_status"] = le.inverse_transform(nn.predict(xtest))

In [43]:
B

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.0,Urban
363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.0,Urban
364,LP002980,Male,No,0,Graduate,No,3250,1993,126.0,360.0,NaN,Semiurban
365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.0,Rural
